In [1]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import pyeto
import spei

In [2]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

DATA_PATH = "./assets/data/GEE_CSV_Exports/"
DATABASE_PATH = "./database/database.db"

### NDVI & EVI & LST

In [3]:
vi_dataset = pd.DataFrame()

# Find *.csv files in the directory with NDVI
list_fles = [x for x in os.listdir(DATA_PATH) if x.endswith(".csv") and ("_NDVI_" in x or "_EVI_" in x or "_LSTNight_" in x or "_LSTDay_" in x)]

for lf in list_fles:
    data = pd.read_csv(filepath_or_buffer=DATA_PATH + lf, na_values=-999)
    data = data[["St_ID", "date", "mean"]]
    data.rename(
        columns={
            "St_ID": "Station_ID",
            "date": "Date",
            "mean": f"{lf.split('_')[2]}_{lf.split('_')[0]}"
        },
        inplace=True
    )
    if vi_dataset.empty:
        vi_dataset = data
    else:
        vi_dataset = vi_dataset.merge(data, on=["Station_ID", "Date"], how="outer")

ndvi_columns = [col for col in vi_dataset.columns if 'NDVI' in col]
evi_columns = [col for col in vi_dataset.columns if 'EVI' in col]
lstday_columns = [col for col in vi_dataset.columns if 'LSTDay' in col]
lstnight_columns = [col for col in vi_dataset.columns if 'LSTNight' in col]

vi_dataset['NDVI'] = vi_dataset[ndvi_columns].mean(axis=1)
vi_dataset['EVI'] = vi_dataset[evi_columns].mean(axis=1)
vi_dataset['LSTDay'] = vi_dataset[lstday_columns].mean(axis=1)
vi_dataset['LSTNight'] = vi_dataset[lstnight_columns].mean(axis=1)
vi_dataset['LST'] = vi_dataset[["LSTDay", "LSTNight"]].mean(axis=1)

vi_dataset['Date'] = pd.to_datetime(vi_dataset['Date']) + pd.offsets.MonthEnd(0)

# Water-Year: 1385-86 to 1402-03
min_date = '2006-09-01' # 18 Years - 24 Stations - Start: 2006-09-23
max_date = '2024-06-30' # End: 2024-09-21
vi_dataset = vi_dataset\
    .query("Date >= @min_date and Date <= @max_date")

vi_dataset.sort_values(by=['Station_ID', 'Date'], inplace=True)
vi_dataset.reset_index(drop=True, inplace=True)

print(vi_dataset.info())

vi_dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Station_ID        5136 non-null   int64         
 1   Date              5136 non-null   datetime64[ns]
 2   NDVI_MOD13A3      5112 non-null   float64       
 3   LSTDay_MOD21C3    5135 non-null   float64       
 4   LSTNight_MYD21C3  5132 non-null   float64       
 5   EVI_MYD13A3       5136 non-null   float64       
 6   LSTNight_MOD21C3  5136 non-null   float64       
 7   NDVI_MYD13A3      5136 non-null   float64       
 8   LSTDay_MYD21C3    5134 non-null   float64       
 9   EVI_MOD13A3       5112 non-null   float64       
 10  NDVI              5136 non-null   float64       
 11  EVI               5136 non-null   float64       
 12  LSTDay            5136 non-null   float64       
 13  LSTNight          5136 non-null   float64       
 14  LST               5136 n

,Station_ID,Date,NDVI_MOD13A3,LSTDay_MOD21C3,LSTNight_MYD21C3,EVI_MYD13A3,LSTNight_MOD21C3,NDVI_MYD13A3,LSTDay_MYD21C3,EVI_MOD13A3,NDVI,EVI,LSTDay,LSTNight,LST
0,40709,2006-09-30,0.5450,29.37,18.67,0.2874,24.09,0.4742,25.87,0.3289,0.50960,0.30815,27.62,21.38,24.500
1,40709,2006-10-31,0.6019,25.09,16.37,0.3065,16.09,0.5955,22.93,0.3294,0.59870,0.31795,24.01,16.23,20.120
2,40709,2006-11-30,0.5557,18.23,9.15,0.2798,15.95,0.5524,16.37,0.2924,0.55405,0.28610,17.30,12.55,14.925
3,40709,2006-12-31,0.4903,10.49,2.97,0.2105,9.99,0.4332,9.81,0.2610,0.46175,0.23575,10.15,6.48,8.315
4,40709,2007-01-31,0.4191,11.37,2.73,0.2021,8.27,0.4171,8.31,0.1875,0.41810,0.19480,9.84,5.50,7.670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,99361,2024-02-29,0.3920,0.11,-1.47,0.1344,2.21,0.2661,14.45,0.1730,0.32905,0.15370,7.28,0.37,3.825
5132,99361,2024-03-31,NaN,5.63,-0.07,0.1551,3.73,0.3203,22.47,NaN,0.32030,0.15510,14.05,1.83,7.940
5133,99361,2024-04-30,0.3901,21.71,2.63,0.5179,7.97,0.7398,26.89,0.2489,0.56495,0.38340,24.30,5.30,14.800
5134,99361,2024-05-31,0.5332,22.71,6.59,0.4374,13.01,0.6308,30.37,0.3797,0.58200,0.40855,26.54,9.80,18.170


In [4]:
conn = sqlite3.connect(DATABASE_PATH)

vi_dataset.to_sql('gee_indices_monthly', conn, if_exists='replace', index=False)
conn.commit()

conn.close()